In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/TFM'

/content/drive/MyDrive/Colab Notebooks/TFM


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import *
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

In [4]:
with open('dataset_X_sliding_window.npy', 'rb') as f:
    X_data = np.load(f, allow_pickle=True)
with open('dataset_y_sliding_window.npy', 'rb') as f:
    y_data = np.load(f, allow_pickle=True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state=42)

In [6]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)
y_test = encoder.transform(y_test)

In [12]:
input_shape = (64, 32)
opt = Adam(0.001)
rnn_model = Sequential()
drp=0.2

rnn_model.add(Input(shape=input_shape))
#rnn_model.add(BatchNormalization())
rnn_model.add(LSTM((64), return_sequences=True))
rnn_model.add(LSTM((128)))
rnn_model.add(Dropout(drp))
rnn_model.add(Dense(3, activation='softmax'))
rnn_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
optimizer=opt,
metrics=["accuracy"])
rnn_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 64, 64)              │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 124,035 (484.51 KB)

 Trainable params: 124,035 (484.51 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_checkpoint_callback = EarlyStopping(monitor='val_accuracy', mode='max', patience = 100, verbose = 1)
history = rnn_model.fit(X_train, y_train, batch_size=32,epochs=1000, callbacks=[model_checkpoint_callback], validation_data=(X_val, y_val))
res = rnn_model.predict(X_test)
print(model_checkpoint_callback.best)
y_pred = np.argmax(res, axis = 1)
print()
print()
p = confusion_matrix(np.argmax(y_test, axis=1), y_pred, labels=[0,1,2])

# list all data in history
print(history.history.keys())

Epoch 1/1000
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 39s 10ms/step - accuracy: 0.3634 - loss: 1.0971 - val_accuracy: 0.3970 - val_loss: 1.0728
Epoch 2/1000
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 37s 10ms/step - accuracy: 0.4041 - loss: 1.0720 - val_accuracy: 0.4091 - val_loss: 1.0626
Epoch 3/1000
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 42s 10ms/step - accuracy: 0.4257 - loss: 1.0523 - val_accuracy: 0.4383 - val_loss: 1.0390
Epoch 4/1000
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 41s 10ms/step - accuracy: 0.4442 - loss: 1.0342 - val_accuracy: 0.4588 - val_loss: 1.0255
Epoch 5/1000
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 46s 12ms/step - accuracy: 0.4705 - loss: 1.0063 - val_accuracy: 0.4786 - val_loss: 0.9948
Epoch 6/1000
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.4896 - loss: 0.9808 - val_accuracy: 0.4935 - val_loss: 0.9759
Epoch 7/1000
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - accuracy: 0.5134 - loss: 0.9480 - val_accuracy: 0.5193 - val_loss: 0.9427
Epoch 8/1000
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - ac